# Iris Classification with TensorFlow Keras

This notebook demonstrates how to classify the Iris dataset using a simple feed-forward neural network built with TensorFlow Keras. TensorBoard is used for visualizing the training progress.


In [ ]:
!pip install tensorflow

In [1]:
import tensorflow as tf
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import os


In [2]:
# Load and preprocess data
iris = load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)

encoder = OneHotEncoder()
y = encoder.fit_transform(y).toarray()

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [3]:
# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu', input_shape=(4,)),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])


d:\113-1\IOT\venv\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [4]:
# Set up TensorBoard
log_dir = "logs/keras_iris"
os.makedirs(log_dir, exist_ok=True)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train the model
model.fit(X_train, y_train, 
          validation_data=(X_test, y_test), 
          epochs=50, 
          callbacks=[tensorboard_callback], 
          verbose=0)


In [5]:
# Evaluate the model
evaluation = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {evaluation[0]}, Test Accuracy: {evaluation[1]}")


Test Loss: 0.34623610973358154, Test Accuracy: 0.8999999761581421


### Visualize TensorBoard
To visualize the training logs, run the following command in your terminal:
```
tensorboard --logdir logs/keras_iris
```


# Iris Classification with PyTorch

This notebook demonstrates how to classify the Iris dataset using a simple feed-forward neural network implemented with PyTorch. TensorBoard is used for visualizing the training progress.


In [ ]:
!pip install torch

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.tensorboard import SummaryWriter
import os


In [3]:
# Load and preprocess data
iris = load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)

encoder = OneHotEncoder()
y = encoder.fit_transform(y).toarray()

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [4]:
# Define the model
class IrisModel(nn.Module):
    def __init__(self):
        super(IrisModel, self).__init__()
        self.fc1 = nn.Linear(4, 10)
        self.fc2 = nn.Linear(10, 10)
        self.fc3 = nn.Linear(10, 3)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.softmax(self.fc3(x), dim=1)
        return x

model = IrisModel()


In [5]:
# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)


In [6]:
# Set up TensorBoard
writer = SummaryWriter("logs/pytorch_iris")

# Training loop
epochs = 50
for epoch in range(epochs):
    model.train()
    for batch in train_loader:
        X_batch, y_batch = batch

        # Forward pass
        outputs = model(X_batch)
        loss = criterion(outputs, torch.argmax(y_batch, dim=1))

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Log training loss
    writer.add_scalar('Loss/train', loss.item(), epoch)

    # Evaluate on test set
    model.eval()
    with torch.no_grad():
        test_loss = 0
        for X_batch, y_batch in test_loader:
            outputs = model(X_batch)
            loss = criterion(outputs, torch.argmax(y_batch, dim=1))
            test_loss += loss.item()

        test_loss /= len(test_loader)
        writer.add_scalar('Loss/test', test_loss, epoch)

writer.close()
print("Training complete. Use TensorBoard to visualize results.")


Training complete. Use TensorBoard to visualize results.


### Visualize TensorBoard
To visualize the training logs, run the following command in your terminal:
```
tensorboard --logdir logs/pytorch_iris
```


# Iris Classification with PyTorch Lightning

This notebook demonstrates how to classify the Iris dataset using a simple feed-forward neural network implemented with PyTorch Lightning. TensorBoard is used for visualizing the training progress.


In [ ]:
!pip install pytorch_lightning

In [8]:
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder


In [10]:
# Load and preprocess data
iris = load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)

encoder = OneHotEncoder()
y = encoder.fit_transform(y).toarray()

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [11]:
# Define Lightning Module
class IrisLightningModel(pl.LightningModule):
    def __init__(self):
        super(IrisLightningModel, self).__init__()
        self.fc1 = nn.Linear(4, 10)
        self.fc2 = nn.Linear(10, 10)
        self.fc3 = nn.Linear(10, 3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x), dim=1)
        return x

    def training_step(self, batch, batch_idx):
        X, y = batch
        outputs = self(X)
        loss = F.cross_entropy(outputs, torch.argmax(y, dim=1))
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        X, y = batch
        outputs = self(X)
        loss = F.cross_entropy(outputs, torch.argmax(y, dim=1))
        self.log('val_loss', loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.01)

# Initialize model
model = IrisLightningModel()


In [12]:
# Initialize TensorBoard logger
logger = pl.loggers.TensorBoardLogger('logs', name='lightning_iris')

# Train the model
trainer = pl.Trainer(max_epochs=50, logger=logger, log_every_n_steps=1)
trainer.fit(model, train_loader, test_loader)

print("Training complete. Use TensorBoard to visualize results.")


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name | Type   | Params | Mode 
----------------------------------------
0 | fc1  | Linear | 50     | train
1 | fc2  | Linear | 110    | train
2 | fc3  | Linear | 33     | train
----------------------------------------
193       Trainable params
0         Non-trainable params
193       Total params
0.001     Total estimated model params size (MB)
3         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

d:\113-1\IOT\venv\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.
d:\113-1\IOT\venv\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.


Training complete. Use TensorBoard to visualize results.


### Visualize TensorBoard
To visualize the training logs, run the following command in your terminal:
```
tensorboard --logdir logs/lightning_iris
```
